In [1]:
import json as j
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score,confusion_matrix,accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

In [2]:
json_data = None
with open('yelp_academic_dataset_review.json') as data_file:
    lines = data_file.readlines()
    joined_lines = "[" + ",".join(lines) + "]"

    json_data = j.loads(joined_lines)

data = pd.DataFrame(json_data)
data.head()

,votes,user_id,review_id,stars,date,text,type,business_id
0,"{'funny': 0, 'useful': 5, 'cool': 2}",rLtl8ZkDX5vH5nAx9C3q5Q,fWKvX83p0-ka4JS3dc6E5A,5,2011-01-26,My wife took me here on my birthday for breakf...,review,9yKzy9PApeiPPOUJEtnvkg
1,"{'funny': 0, 'useful': 0, 'cool': 0}",0a2KyEL0d3Yb1V6aivbIuQ,IjZ33sJrzXqU-0X6U8NwyA,5,2011-07-27,I have no idea why some people give bad review...,review,ZRJwVLyzEJq1VAihDhYiow
2,"{'funny': 0, 'useful': 1, 'cool': 0}",0hT2KtfLiobPvh6cDC8JQg,IESLBzqUCLdSzSqm0eCSxQ,4,2012-06-14,love the gyro plate. Rice is so good and I als...,review,6oRAC4uyJCsJl1X0WZpVSA
3,"{'funny': 0, 'useful': 2, 'cool': 1}",uZetl9T0NcROGOyFfughhg,G-WvGaISbqqaMHlNnByodA,5,2010-05-27,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,_1QQZuf4zZOyFCvXc0o6Vg
4,"{'funny': 0, 'useful': 0, 'cool': 0}",vYmM4KTsC8ZfQBg-j5MWkw,1uJFq2r5QfJG_6ExMRCaGw,5,2012-01-05,General Manager Scott Petello is a good egg!!!...,review,6ozycU1RpktNG2-1BroVtw


In [3]:
data['stars'].value_counts()

4    79878
5    76193
3    35363
2    20957
1    17516
Name: stars, dtype: int64

In [4]:
data = data[data.stars != 3]
data['sentiment'] = data['stars'] >= 4

In [5]:
data.head()

,votes,user_id,review_id,stars,date,text,type,business_id,sentiment
0,"{'funny': 0, 'useful': 5, 'cool': 2}",rLtl8ZkDX5vH5nAx9C3q5Q,fWKvX83p0-ka4JS3dc6E5A,5,2011-01-26,My wife took me here on my birthday for breakf...,review,9yKzy9PApeiPPOUJEtnvkg,True
1,"{'funny': 0, 'useful': 0, 'cool': 0}",0a2KyEL0d3Yb1V6aivbIuQ,IjZ33sJrzXqU-0X6U8NwyA,5,2011-07-27,I have no idea why some people give bad review...,review,ZRJwVLyzEJq1VAihDhYiow,True
2,"{'funny': 0, 'useful': 1, 'cool': 0}",0hT2KtfLiobPvh6cDC8JQg,IESLBzqUCLdSzSqm0eCSxQ,4,2012-06-14,love the gyro plate. Rice is so good and I als...,review,6oRAC4uyJCsJl1X0WZpVSA,True
3,"{'funny': 0, 'useful': 2, 'cool': 1}",uZetl9T0NcROGOyFfughhg,G-WvGaISbqqaMHlNnByodA,5,2010-05-27,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,_1QQZuf4zZOyFCvXc0o6Vg,True
4,"{'funny': 0, 'useful': 0, 'cool': 0}",vYmM4KTsC8ZfQBg-j5MWkw,1uJFq2r5QfJG_6ExMRCaGw,5,2012-01-05,General Manager Scott Petello is a good egg!!!...,review,6ozycU1RpktNG2-1BroVtw,True


In [6]:
data['sentiment'].value_counts()

True     156071
False     73836
Name: sentiment, dtype: int64

In [7]:
X = data['text'][:70000]
Y = data.sentiment[:70000]
print([X.shape, Y.shape])

[(70000,), (70000,)]


In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size = 0.3, random_state=42)

In [9]:
count = CountVectorizer()
temp = count.fit_transform(X_train)
tfid = TfidfTransformer()
X_train_temp = tfid.fit_transform(temp)

In [10]:
text_clf = LogisticRegression()
model = text_clf.fit(X_train_temp, Y_train)

In [11]:
Y_pred = model.predict(tfid.transform(count.transform(X_test)))
print(np.mean(Y_pred == Y_test)*100)

86.14761904761905


In [12]:
# Make Predictions

print("Total No. of Observations : " + str(len(Y_test.values)))
print("Positive Observations : " + str(sum(Y_test.values)))
print("Negative Observations : " + str(len(Y_test.values) - sum(Y_test.values)))

print("Majority Class is : " + str((sum(Y_test.values)/len(Y_test.values))*100) + "%")

Total No. of Observations : 21000
Positive Observations : 14342
Negative Observations : 6658
Majority Class is : 68.29523809523809%


In [13]:
print("confusion matrix: \n" + str(confusion_matrix(Y_test, Y_pred)))
print("accuracy score: {0:.2f}".format((accuracy_score(Y_test, Y_pred)*100)),"%")
print("precision score: {0:.2f}".format((precision_score(Y_test, Y_pred)*100)),"%")
print("recall score: {0:.2f}".format((recall_score(Y_test, Y_pred)*100)),"%")
print("f1-score: {0:.2f}".format((f1_score(Y_test, Y_pred)*100)),"%")

confusion matrix: 
[[ 4728  1930]
 [  979 13363]]
accuracy score: 86.15 %
precision score: 87.38 %
recall score: 93.17 %
f1-score: 90.18 %


In [14]:
# Using PIPELINE
from sklearn.pipeline import Pipeline

In [15]:
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('classifier', LogisticRegression())
])
# Using K-Fold
from sklearn.model_selection import cross_val_score
scores = cross_val_score(pipeline,X_train,Y_train,cv=5,n_jobs=-1,scoring='accuracy')
mean_score = scores.mean()
std_score = scores.std()
print(mean_score)
print(std_score)

print(pipeline.get_params())


0.8574693877551021
0.004023773583660189
{'memory': None, 'steps': [('vectorizer', CountVectorizer()), ('tfidf', TfidfTransformer()), ('classifier', LogisticRegression())], 'verbose': False, 'vectorizer': CountVectorizer(), 'tfidf': TfidfTransformer(), 'classifier': LogisticRegression(), 'vectorizer__analyzer': 'word', 'vectorizer__binary': False, 'vectorizer__decode_error': 'strict', 'vectorizer__dtype': <class 'numpy.int64'>, 'vectorizer__encoding': 'utf-8', 'vectorizer__input': 'content', 'vectorizer__lowercase': True, 'vectorizer__max_df': 1.0, 'vectorizer__max_features': None, 'vectorizer__min_df': 1, 'vectorizer__ngram_range': (1, 1), 'vectorizer__preprocessor': None, 'vectorizer__stop_words': None, 'vectorizer__strip_accents': None, 'vectorizer__token_pattern': '(?u)\\b\\w\\w+\\b', 'vectorizer__tokenizer': None, 'vectorizer__vocabulary': None, 'tfidf__norm': 'l2', 'tfidf__smooth_idf': True, 'tfidf__sublinear_tf': False, 'tfidf__use_idf': True, 'classifier__C': 1.0, 'classifier__c